# Image Calib

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
%matplotlib qt



# Prepare object point
nx = 9 # number of inside corners in x
ny = 6 # number of inside corners in y

# Arrays to store object points and image points from all the images
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny*nx, 3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2) # adding x, y coordinates at firt two columns

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Read in and make a list of calibration images # Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    # read in each image
    img = cv2.imread(fname)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None) # Gray scale and ny by nx corners dimenstions of chess board and flag for None

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret) # Draw chess corner on image
#         print(len(objpoints))
        ## Begin Distortion:
        # Begin calibrate camera:
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
        # undistort images with given mtx, dist at the image
        undistorted = cv2.undistort(img, mtx, dist, None, mtx)
        
        
        ## Begin warper
        img_size = gray.shape[::-1]# (gray.shape[1], gray.shape[0])
        offset = 100 # offset for dst pts

        # Perspective transform:
        # Four source coordinates:
        #tl, tr, bl, br
        src = np.float32([corners[0], corners[nx-1], corners[-nx], corners[-1]])

        # Four desired coordinates:
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [offset, img_size[1]-offset],
                                     [img_size[0]-offset, img_size[1]-offset]])

        # Compute the perspective transform, M
        M = cv2.getPerspectiveTransform(src, dst)
        
        # Create wapred image - uses linear interpolation
        warped_img = cv2.warpPerspective(img, M, img_size)

        # Draw and display the corners
        cv2.imshow('img',warped_img)
#             plt.imshow(warped_img)
#             plt.imsave(warped_img)
        cv2.waitKey(500)



        # Save image
        folderName = '../HC_camera_cal_output/'
        isFile = os.path.exists(folderName)
        if isFile:
            plt.imsave(os.path.join(folderName+fname[17::]), warped_img)
#                 plt.savefig(os.path.join(folderName+fname[17::]))
            # cv2.imread(os.path.join(folderName+fname[17::]),int(img))
            # plt.imsave('image_new.jpg',image )
        else:
            os.makedirs(folderName)
            plt.imsave(os.path.join(folderName+fname[17::]), warped_img)
#                 plt.savefig(os.path.join(folderName+fname[17::]))
#             cv2.imread(os.path.join(folderName+fname[17::]),int(img))

    
cv2.destroyAllWindows()


# Lane Detect

In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
# %matplotlib qt
%matplotlib inline



def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray =img
    # Calculate directional gradient
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.sqrt(np.square(sobelx))
    abs_sobely = np.sqrt(np.square(sobely))
    abs_sobelxy = np.sqrt(np.square(sobelx)+np.square(sobely))

    if orient == 'x':
        scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    else:
        scaled_sobel = np.uint8(255*abs_sobely/np.max(abs_sobely))


    # Apply threshold
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1


    return grad_binary


def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return mag_binary



def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     gray = img
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.sqrt(np.square(sobelx))
    abs_sobely = np.sqrt(np.square(sobely))
    abs_sobelxy = np.sqrt(np.square(sobelx)+np.square(sobely))

    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    theta = np.arctan2(abs_sobely, abs_sobelx)

#     normalize
#     theta = np.uint8(theta/(np.max(theta)/255))

    scaled_theta = np.uint8(theta/np.max(theta/255))

    # 5) Create a binary mask where direction thresholds are met
#     binary_output = np.zeros_like(theta)
    dir_binary = np.zeros_like(scaled_theta)
    dir_binary[(theta >= thresh[0]) & (theta <= thresh[1])] = 1
    return dir_binary



def hist(image):
    bottom_half = image[image.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    return histogram


def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 12
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img, leftx_base, rightx_base, midpoint


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img, leftx_base, rightx_base, midpoint = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
# 
    # Plots the left and right polynomials on the lane lines
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
#     print(leftx.size)
#     print(rightx.size)

    return out_img, ploty, left_fit, right_fit, leftx, lefty, rightx, righty, leftx_base, rightx_base, midpoint


def measure_curvature_pixels(ploty, left_fit, right_fit):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
#     ploty, left_fit, right_fit = generate_data()
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

    return left_curverad, right_curverad


def measure_curvature_real(ploty, left_fit, right_fit, leftx, lefty, rightx, righty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Make sure to feed in your real data instead in your project!
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
#     ploty, left_fit_cr, right_fit_cr = generate_data(ym_per_pix, xm_per_pix)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    y_left_eval = np.max(lefty)
    y_right_eval = np.max(righty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    # left_curverad = 0  ## Implement the calculation of the left line here
    # right_curverad = 0  ## Implement the calculation of the right line here
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
#     left_curverad = ((1 + (2*left_fit_cr[0]*y_left_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
#     right_curverad = ((1 + (2*right_fit_cr[0]*y_right_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    avg_curv = np.average([left_curverad, right_curverad])
    
#     return left_curverad, right_curverad
#     return 'Left_R: {:.2f} m, Right_R:{:.2f} m'.format(left_curverad, right_curverad)
    return 'Curvature Radius: {:.2f} m'.format(avg_curv)


def real_mid_offset(leftx_base, rightx_base, midpoint):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    current_lane_midpoint = np.average([leftx_base, rightx_base])
    mid_offset = current_lane_midpoint - midpoint
    real_mid_offset = mid_offset * xm_per_pix
    
    if real_mid_offset >= 0: return 'Center Offset: {:.2f} m Left of Center'.format(real_mid_offset)
    else: return 'Center Offset: {:.2f} m Right of Center'.format(np.abs(real_mid_offset))
    
    
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped,left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    # Plot the polynomial lines onto the image
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##

    return result, left_fitx, right_fitx, ploty, out_img





def process_image(image):
#
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)


    # RGB
    # R_RGB_thresh = (218, 255)# N
    R_RGB = image[:, :, 0]
    G_RGB = image[:, :, 1]
    B_RGB = image[:, :, 2]

    R_RGB_thresh = (218, 255)# N
    R_binary = np.zeros_like(R_RGB)
    R_binary[(R_RGB <= R_RGB_thresh[1]) & (R_RGB >= R_RGB_thresh[0])] = 1
    R_binary_RGB = R_binary


    G_RGB_thresh = (185, 255)
    G_binary = np.zeros_like(G_RGB)
    G_binary[(G_RGB <= G_RGB_thresh[1]) & (G_RGB >= G_RGB_thresh[0])] = 1 ## <<-------- Interfere with R binary selection
    G_binary_RGB = G_binary

    B_RGB_thresh = (46, 68)
    B_binary = np.zeros_like(B_RGB)
    B_binary[(B_RGB <= B_RGB_thresh[1]) & (B_RGB >= B_RGB_thresh[0])] = 1 ## <<-------- Interfere with R binary selection
    B_binary_RGB = B_binary



    #LAB
    brightLAB = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L_LAB = np.abs(brightLAB[:,:,0])/np.max(np.abs(brightLAB[:,:,0]))
    A_LAB = np.abs(brightLAB[:,:,1])/np.max(np.abs(brightLAB[:,:,1]))# <-------8,7,5 ,4l
    B_LAB = np.abs(brightLAB[:,:,2])/np.max(np.abs(brightLAB[:,:,2]))# <-------8,7, 5, 4l
    #     L_thresh = (.83,1)
    #     L_binary = np.zeros_like(L)
    #     L_binary[(L <= L_thresh[1]) & (L >= L_thresh[0])] = 1
    #     L_binary_LAB = L_binary
    # A_LAB_thresh = (0,.71)
    A_LAB_thresh = (0,.61)
    A_binary = np.zeros_like(A_LAB)
    A_binary[(A_LAB <= A_LAB_thresh[1]) & (A_LAB >= A_LAB_thresh[0])] = 1
    A_binary_LAB = A_binary

    B_LAB_thresh = (0,.60)
    B_binary = np.zeros_like(B_LAB)
    B_binary[(B_LAB <= B_LAB_thresh[1]) & (B_LAB >= B_LAB_thresh[0])] = 1
    # B_binary_LAB = B_binary

    B_binary_LAB = abs_sobel_thresh(B_binary, orient='x', sobel_kernel=7, thresh=(25, 255)) #<--------added Apr8-#<---------



    # HLS

    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H_HLS = hls[:,:,0]
    L_HLS = hls[:,:,1]
    S_HLS = hls[:,:,2]
    S_HLS_thresh = (190,255)
    S_binary = np.zeros_like(S_HLS)
    S_binary[(S_HLS <= S_HLS_thresh[1]) & (S_HLS >= S_HLS_thresh[0])] = 1
    S_binary_HLS = S_binary



    #HSV

    brightHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    H_HSV = brightHSV[:,:,0]
    S_HSV = brightHSV[:,:,1]# <-------8,5
    V_HSV = brightHSV[:,:,2]#7,6

    S_HSV_thresh = (120,255)
    S_binary = np.zeros_like(S_HSV)
    S_binary[(S_HSV <= S_HSV_thresh[1]) & (S_HSV >= S_HSV_thresh[0])] = 1
    S_binary_HSV = S_binary

    #     V_thresh = (.82,1)
    V_HSV_thresh = (200,255)
    V_binary = np.zeros_like(V_HSV)
    V_binary[(V_HSV <= V_HSV_thresh[1]) & (V_HSV >= V_HSV_thresh[0])] = 1
    V_binary_HSV = V_binary

    V_HSV_thresh2 = (0, 194)
    # V_HSV_thresh2 = (0, 208)
    V_binary_HSV2 = np.zeros_like(V_HSV)
    V_binary_HSV2[(V_HSV >= V_HSV_thresh2[0]) & (V_HSV <= V_HSV_thresh2[1])] = 1


    #YCRB
    ycrb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    Y_YCRB = ycrb[:,:,0]#6RL, 4RL
    Cr_YCRB = ycrb[:,:,1]# <-------8,7,5
    Cb_YCRB = ycrb[:,:,2]# <-------8,5


    Cr_YCRB_thresh = (0, 102)
    Cr_binary = np.zeros_like(Cr_YCRB)
    Cr_binary[(Cr_YCRB <= Cr_YCRB_thresh[1]) & (Cr_YCRB >= Cr_YCRB_thresh[0])] = 1
    Cr_binary_YCRB = Cr_binary
    #     Cb_thresh = (.56,1)

    Cb_YCRB_thresh = (164, 255)
    Cb_binary = np.zeros_like(Cb_YCRB)
    Cb_binary[(Cb_YCRB <= Cb_YCRB_thresh[1]) & (Cb_YCRB >= Cb_YCRB_thresh[0])] = 1
    Cb_binary_YCRB = Cb_binary



    # combined
    # B_binary_LAB <<<<------LAB 
    # V_binary_HSV <<<<<<<<<<<<<DELETE THIS WE WILL NOT USING THIS

    # S_binaryHSV  <<<<<<<<<<<<< DELETE THIS WE WILL NOT USING THIS
    # Cb_binary_YCRB

    # Cr_binary_YCRB
    # R_binary_RGB <----RGB

    # A_binary_LAB <<<<------LAB
    # S_binary_HLS  <<<<----







    # combined
    # B_binary_LAB <<<<------LAB 
    # Cb_binary_YCRB

    # Cr_binary_YCRB
    # R_binary_RGB <----RGB

    # A_binary_LAB <<<<------LAB
    # S_binary_HLS  <<<<----

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    #1
    # B_binary_LAB
    # Cb_binary_YCRB
    color_binary1 = np.dstack((np.zeros_like(B_binary_LAB), B_binary_LAB, Cb_binary_YCRB)) * 255
    combined_binary1 = np.zeros_like(Cb_binary_YCRB)
    combined_binary1[(B_binary_LAB == 1) & (Cb_binary_YCRB == 1)] = 1  

    #2
    # Cr_binary_YCRB
    # R_binary_RGB <----RGB
    color_binary2 = np.dstack((np.zeros_like(R_binary_RGB), R_binary_RGB, Cr_binary_YCRB)) *255
    combined_binary2 = np.zeros_like(Cr_binary_YCRB)
    combined_binary2[(R_binary_RGB == 1) | (Cr_binary_YCRB == 1)] = 1

    #3 
    # A_binary_LAB <<<<------LAB
    # S_binary_HLS  <<<<---- AND S_binary_HLS
    color_binary3 = np.dstack((np.zeros_like(A_binary_LAB), A_binary_LAB, S_binary_HLS)) *255
    combined_binary3 = np.zeros_like(S_binary_HLS)
    combined_binary3[((A_binary_LAB == 1) & (B_binary_LAB==1)) & (S_binary_HLS == 1)] = 1

    # #4 
    # color_binary4 = np.dstack((np.zeros_like(A_binary_LAB), A_binary_LAB, B_binary_RGB))
    # combined_binary4 = np.zeros_like(B_binary_RGB)
    # combined_binary4[(A_binary_LAB == 1) | (B_binary_RGB == 1)] = 1

    # ----------


    # combined 1 and 2
    color_binary_1n2 = np.dstack((np.zeros_like(combined_binary1), combined_binary1, combined_binary2)) * 255
    combined_binary_1n2 = np.zeros_like(combined_binary2)
    combined_binary_1n2[(combined_binary1 == 1) | (combined_binary2 == 1)] = 1

    # # Combine 3 and 4!
    # color_binary_3n4 = np.dstack((np.zeros_like(combined_binary3), combined_binary3, combined_binary4))
    # combined_binary_3n4 = np.zeros_like(combined_binary4)
    # combined_binary_3n4[(combined_binary3 == 1) | (combined_binary4 == 1)] = 1

    #Final 1n2 and 3
    color_binary_f = np.dstack((np.zeros_like(combined_binary_1n2), combined_binary_1n2, combined_binary3)) * 255
    combined_binary_f = np.zeros_like(combined_binary3)
    combined_binary_f[(combined_binary_1n2 == 1) | (combined_binary3 == 1)] = 1


    # ----------------------

    # Special
    Sx_binary1 = abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(25, 255))
    Sx_binary = abs_sobel_thresh(Sx_binary1, orient='x', sobel_kernel=7, thresh=(25, 255)) #<--------Added apr92021-#<---------
    combined_test = np.zeros_like(Sx_binary)
    combined_test[(Sx_binary == 1) & (V_binary_HSV2 != 1)] = 1
    # plt.imshow(combined_test)



    # COMBINED2
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    # color_binary_a1a2 = np.dstack(( np.zeros_like(a3), a3, Sx_binary)) * 255

    # # Combine the two binary thresholds
    # combined_binary_a1a2 = np.zeros_like(a3)
    # # combined_binary_a1a2[(Sx_binary == 1) | (a3 ==1)] = 1
    # combined_binary_a1a2[(Sx_binary == 1)] = 1

    # combined_binary_a1a2 = Sx_binary
    combined_binary_special = combined_test


    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary_final = np.dstack((np.zeros_like(combined_binary_f), combined_binary_f, combined_binary_special)) * 255

    # Combine the two binary thresholds
    combined_binary_final = np.zeros_like(combined_binary_f)
    combined_binary_final[(combined_binary_f == 1) | (combined_binary_special == 1)] = 1
    # combined_binary_final = combined_binary_f


    ########################





    # ############


    #     # Stack each channel to view their individual contributions in green and blue respectively
    #     # This returns a stack of the two binary images, whose components you can see as different colors
    #     color_binary = np.dstack(( np.zeros_like(best_binary1), best_binary1, best_binary2)) * 255

    #     # Combine the two binary thresholds
    #     combined_binary = np.zeros_like(best_binary1)
    #     combined_binary[(best_binary2 == 1) | (best_binary1 == 1)] = 1
    #     best_final = combined_binary



    # best_binary1 = combined_binary
    #     best_final = combined_binary

    img_size = (image.shape[1], image.shape[0])

    src = np.float32(
        [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
        [((img_size[0] / 6) - 10), img_size[1]],
        [(img_size[0] * 5 / 6) + 60, img_size[1]],
        [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
    dst = np.float32(
        [[(img_size[0] / 4), 0],
        [(img_size[0] / 4), img_size[1]],
        [(img_size[0] * 3 / 4), img_size[1]],
        [(img_size[0] * 3 / 4), 0]])

    M = cv2.getPerspectiveTransform(src, dst)
    warped2 = cv2.warpPerspective(image, M, img_size, flags=cv2.INTER_LINEAR)
    # trythis = warped.copy()
    trythis2 = image

    warped = cv2.warpPerspective(combined_binary_final, M, img_size, flags=cv2.INTER_LINEAR)

    img = warped/255
    histogram = hist(img)
    # plt.plot(histogram)

    binary_warped = warped


    out_img, ploty,left_fit, right_fit, leftx, lefty, rightx, righty, leftx_base, rightx_base, midpoint = fit_polynomial(binary_warped)
    out_img2 = out_img # Draw line ine squuare box


    # Run image through the pipeline
    # Note that in your project, you'll also want to feed in the previous fits
    result, left_fitx, right_fitx, ploty, out_img = search_around_poly(binary_warped, left_fit, right_fit)

    # %matplotlib qt
    # print(type(warped))
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    #     img_size = (warped.shape[1], warped.shape[0])



    Minv = cv2.getPerspectiveTransform(dst, src)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0]))

    warp2 = cv2.warpPerspective(warped2, Minv, (image.shape[1], image.shape[0]))
    real_mid_off = real_mid_offset(leftx_base, rightx_base, midpoint)
    curv_R = measure_curvature_real(ploty, left_fit, right_fit, leftx, lefty, rightx, righty)

    return newwarp, real_mid_off, curv_R

def process_image_end(img):
    image = img
    result_a, real_mid_off, curv_R = process_image(image)
    result = cv2.addWeighted(image, 1, result_a, 0.3, 0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, real_mid_off,(10,100), font, 1.5,(0,255,0),3,cv2.LINE_AA)
    cv2.putText(result, curv_R ,(10,50), font, 1.5,(0,255,0),3,cv2.LINE_AA)

    return result

# fileName = '../test_images/test5.jpg'
# image = mpimg.imread(fileName)

# result =process_image_end(image)
# # result_a =  process_image(image)
# # result = cv2.addWeighted(image, 1, result_a, 0.3, 0)

# plt.imshow(result)
# # process_image_end




In [3]:
white_output = '../HC_video_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip1 = VideoFileClip("../project_video.mp4").subclip(0,3)
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image_end) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video ../HC_video_output/project_video.mp4.
Moviepy - Writing video ../HC_video_output/project_video.mp4



Moviepy - Done !
Moviepy - video ready ../HC_video_output/project_video.mp4
CPU times: user 30min 54s, sys: 1min 39s, total: 32min 34s
Wall time: 3min 37s


In [4]:
# white_output = '../HC_video_output/project_video.mp4'

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [5]:
white_output2 = '../HC_video_output/challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip1 = VideoFileClip("../project_video.mp4").subclip(22,25)
clip2 = VideoFileClip("../challenge_video.mp4")
# clip2 = VideoFileClip("../challenge_video.mp4").subclip(0,5)
white_clip = clip2.fl_image(process_image_end) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output2, audio=False)

t:   0%|          | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video ../HC_video_output/challenge_video.mp4.
Moviepy - Writing video ../HC_video_output/challenge_video.mp4



t:  28%|██▊       | 134/485 [00:22<00:58,  6.05it/s, now=None]

TypeError: expected non-empty vector for x

In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output2))

In [7]:
white_output3 = '../HC_video_output/harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip1 = VideoFileClip("../project_video.mp4").subclip(22,25)
clip3 = VideoFileClip("../harder_challenge_video.mp4")
# clip3 = VideoFileClip("../harder_challenge_video.mp4").subclip(0,5)

white_clip = clip3.fl_image(process_image_end) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output3, audio=False)

t:   0%|          | 2/1199 [00:00<01:50, 10.81it/s, now=None]

Moviepy - Building video ../HC_video_output/harder_challenge_video.mp4.
Moviepy - Writing video ../HC_video_output/harder_challenge_video.mp4




t:  49%|████▉     | 591/1199 [01:58<01:48,  5.58it/s, now=None]

TypeError: expected non-empty vector for x

In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output3))